# Getting started

This script will explore [Elena Cuoco's script](https://www.kaggle.com/elenacuoco/grasp-and-lift-eeg-detection/simple-grasp-with-sklearn) in an effort to understand what is going on.

In [1]:
# contains important preprocessing tools to read in our data
from tools import *
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

### Simple Logistic Regression

In [27]:
# will hold all ids and all predictions
all_ids = []
all_predictions = []

subsample = 100
num_subjects = 13
num_series = 9
human_labels = ['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']

In [28]:
# we go with 5 samples first

for subject_id in range(1,num_subjects):
    y_raw = []
    raw = []
    
    # read in training data
    for series_id in range(1,num_series):
        data,labels = prepare_data_train(subject_id, series_id)
        raw.append(data)
        y_raw.append(labels)
    
    
    # concatanenate the data sets into one dataframe
    X = pd.concat(raw)
    y = pd.concat(y_raw)
    
    X_train = np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))
    
    
    test = []
    idx = [] # test data ids
    
    # holds the test data
    for series_id in range(9,11):
        data = prepare_data_test(subject_id, series_id)
        test.append(data)
        print 'size of current dataset: subject=',subject_id,', series=',series_id,' is ',data.shape
        idx.append(np.array(data['id']))
    
    X_test = pd.concat(test)
    all_ids.append(np.concatenate(idx))
    X_test = X_test.drop(['id'], axis=1)
    X_test = np.asarray(X_test.astype(float))
    
    
    lr = LogisticRegression()
    current_prediction = np.empty((X_test.shape[0], 6)) # number of test samples X number of labels
    
    X_train = data_preprocess_train(X_train)
    X_test = data_preprocess_test(X_test)
    
    # train the classifier for each label
    for i in range(6):
        print 'Training subject_id = ',subject_id, ' label: ',human_labels[i]
        y_train = y[:,i]
        lr.fit(X_train[::subsample,:], y_train[::subsample])
        current_prediction[:,i] = lr.predict_proba(X_test)[:,1]
    
    all_predictions.append(current_prediction)
    


size of current dataset: subject= 1 , series= 9  is  (115953, 33)
size of current dataset: subject= 1 , series= 10  is  (117128, 33)
Training subject_id =  1  label:  HandStart
Training subject_id =  1  label:  FirstDigitTouch
Training subject_id =  1  label:  BothStartLoadPhase
Training subject_id =  1  label:  LiftOff
Training subject_id =  1  label:  Replace
Training subject_id =  1  label:  BothReleased
size of current dataset: subject= 2 , series= 9  is  (147373, 33)
size of current dataset: subject= 2 , series= 10  is  (150712, 33)
Training subject_id =  2  label:  HandStart
Training subject_id =  2  label:  FirstDigitTouch
Training subject_id =  2  label:  BothStartLoadPhase
Training subject_id =  2  label:  LiftOff
Training subject_id =  2  label:  Replace
Training subject_id =  2  label:  BothReleased
size of current dataset: subject= 3 , series= 9  is  (111945, 33)
size of current dataset: subject= 3 , series= 10  is  (113394, 33)
Training subject_id =  3  label:  HandStart
T

In [29]:
np.concatenate(all_ids).shape

(3144171,)

In [30]:
np.concatenate(all_predictions).shape

(3144171, 6)

In [31]:
submission_file = 'submission1.csv'

submission = pd.DataFrame(index=np.concatenate(all_ids), \
                         columns = human_labels, \
                         data = np.concatenate(all_predictions))

submission.to_csv(submission_file, index_label='id', float_format='%.3f')